In [4]:
# "111"
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# ==============================================================================
# 模块 1: 元胞自动机模拟器 (来自你的代码，无需修改)
# ==============================================================================

class ElementaryCellularAutomaton:
    """
    一个初等元胞自动机（ECA）的模拟器。
    """
    def __init__(self, rule_number: int, size: int = 100):
        if not 0 <= rule_number <= 255:
            raise ValueError("规则编号必须在0到255之间。")
        self.rule_number = rule_number
        self.size = size
        self.rule_table = np.array([int(x) for x in f'{rule_number:08b}'], dtype=np.uint8)
        self.state = None
        self.history = []

    def initialize(self, initial_state: np.ndarray = None):
        if initial_state is None:
            self.state = np.random.randint(0, 2, self.size, dtype=np.uint8)
        else:
            if initial_state.shape != (self.size,):
                raise ValueError(f"初始状态的形状必须为({self.size},)。")
            self.state = initial_state.astype(np.uint8)
        self.history = [self.state.copy()]

    def evolve(self, steps: int = 1):
        if self.state is None:
            raise RuntimeError("必须先调用initialize()方法来初始化状态。")
        for _ in range(steps):
            left_neighbors = np.roll(self.state, 1)
            right_neighbors = np.roll(self.state, -1)
            indices = 4 * left_neighbors + 2 * self.state + 1 * right_neighbors
            self.state = self.rule_table[7 - indices]
            self.history.append(self.state.copy())

# ==============================================================================
# 模块 2: 时间序列分析 (最终修正版)
# ==============================================================================

def detect_periodicity(ts: np.ndarray, min_period: int = 2, significance: float = 12.0) -> int | None:
    """
    使用快速傅里叶变换(FFT)检测时间序列中的主周期。
    显著性(significance)参数被进一步提高，以确保只捕捉非常强的周期信号。
    """
    n = len(ts)
    if n < min_period * 2:
        return None

    ts = ts - np.mean(ts)
    if np.all(np.abs(ts) < 1e-6):
        return 1

    fft_power = np.abs(np.fft.rfft(ts))**2
    fft_freq = np.fft.rfftfreq(n)
    
    fft_power[fft_freq > 1.0 / min_period] = 0
    fft_power[0] = 0 

    if np.all(fft_power < 1e-9):
        return None

    peak_idx = np.argmax(fft_power)
    peak_power = fft_power[peak_idx]

    # 检查峰值是否足够显著
    mean_power = (np.sum(fft_power) - peak_power) / (len(fft_power) - 1)
    if mean_power < 1e-9 or peak_power / mean_power < significance:
        return None

    dominant_freq = fft_freq[peak_idx]
    if dominant_freq < 1e-9:
        return None
    period = int(round(1 / dominant_freq))

    if period > n // 2:
        return None

    return period

def is_transient(ts: np.ndarray, tail_len: int = 400, var_ratio: float = 5.0) -> bool:
    """
    更严格地检查瞬态混沌。
    """
    if len(ts) < tail_len * 2:
        return False

    tail = ts[-tail_len:]
    head = ts[:-tail_len]
    
    std_tail = np.std(tail)
    std_head = np.std(head)

    # 条件1: 头部必须比尾部混乱得多
    if std_tail < 1e-6: # 尾部完全稳定
        if std_head < 2.0: return False # 头部不够乱
    elif (std_head / std_tail) < var_ratio:
        return False
    
    # 条件2: 尾部本身必须是简单的（稳定的或短周期的）
    period_in_tail = detect_periodicity(tail, significance=8.0)
    if period_in_tail is None and std_tail > 2.0:
        return False

    # 条件3: 整个序列不应被简单地归类为一个稳定的长周期信号
    if detect_periodicity(ts, significance=10.0) is not None:
        return False

    return True

def characterize_series(ts: np.ndarray, L: int, tail_len: int = 400) -> str:
    """
    最终版的分类函数，采用更鲁棒的分层决策逻辑。
    """
    # 1. 处理最简单的情况
    if np.all(ts == 0):
        return '1'
    
    # 2. 优先检查最独特的行为：瞬态混沌
    if is_transient(ts, tail_len=tail_len):
        return 'T'
        
    # 3. 其次，寻找全局的、最强的证据：整个序列的周期性
    period = detect_periodicity(ts, significance=12.0)
    if period is not None:
        if period <= 20:
            return 'LP'
        if 20 < period < 5 * L:
            return 'HP'
        if period >= 5 * L:
            return 'U'
            
    # 4. 如果既非瞬态也无全局周期，再用尾部稳定性来区分 LP 和 C
    if len(ts) > tail_len:
        tail = ts[-tail_len:]
        if np.std(tail) < 2.0:
            return 'LP' # 这是一个稳定下来但被FFT错过的简单规则

    # 5. 如果以上所有情况都不是，那它就是混沌
    return 'C'

# ==============================================================================
# 模块 3: 模拟与分类流程
# ==============================================================================

COMPLEXITY_HIERARCHY = {'T': 5, 'U': 4, 'C': 3, 'HP': 2, 'LP': 1, '1': 0}

def hamming_distance(arr1: np.ndarray, arr2: np.ndarray) -> int:
    return np.count_nonzero(arr1 != arr2)

def run_single_trial(rule_number: int, size: int, duration: int, transient_period: int) -> np.ndarray:
    eca_A = ElementaryCellularAutomaton(rule_number, size)
    eca_A.initialize()
    
    eca_B = ElementaryCellularAutomaton(rule_number, size)
    eca_B.initialize(initial_state=eca_A.state.copy())

    eca_A.evolve(steps=transient_period)
    eca_B.evolve(steps=transient_period)
    
    perturb_idx = size // 2
    eca_B.state[perturb_idx] = 1 - eca_B.state[perturb_idx]

    hamming_ts = []
    for _ in range(duration):
        eca_A.evolve()
        eca_B.evolve()
        dist = hamming_distance(eca_A.state, eca_B.state)
        hamming_ts.append(dist)
        
    return np.array(hamming_ts)

def classify_rule(rule_number: int, num_trials: int = 100, size: int = 100, 
                  duration: int = 3000, transient_period: int = 15) -> str:
    results = []
    for _ in tqdm(range(num_trials), desc=f"Classifying Rule {rule_number}", leave=False, ncols=100):
        ts = run_single_trial(rule_number, size, duration, transient_period)
        classification = characterize_series(ts, size)
        results.append(classification)
    
    if not results:
        return "Error"
        
    most_complex_result = max(results, key=lambda r: COMPLEXITY_HIERARCHY.get(r, -1))
    
    return most_complex_result

# ==============================================================================
# 模块 4: 复现与验证
# ==============================================================================

def verify_all_rules_from_paper():
    """
    根据论文中提供的所有唯一规则来验证分类算法。
    """
    paper_results_full = {
        0: '1', 1: 'LP', 2: 'LP', 3: 'LP', 4: 'LP', 5: 'LP', 6: 'LP', 7: 'LP',
        8: '1', 9: 'LP', 10: 'LP', 11: 'LP', 12: 'LP', 13: 'LP', 14: 'LP',
        15: 'LP', 18: 'C', 19: 'LP', 22: 'C', 23: 'LP', 24: 'LP', 25: 'HP',
        26: 'LP', 27: 'LP', 28: 'LP', 29: 'LP', 30: 'C', 32: '1', 33: 'LP',
        34: 'LP', 35: 'LP', 36: 'LP', 37: 'LP', 38: 'LP', 40: '1', 41: 'T',
        42: 'LP', 43: 'LP', 44: 'LP', 45: 'C', 46: 'LP', 50: 'LP', 51: 'LP',
        54: 'T', 56: 'LP', 57: 'LP', 58: 'LP', 60: 'U', 62: 'HP', 72: 'LP',
        73: 'T', 74: 'LP', 76: 'LP', 77: 'LP', 78: 'LP', 90: 'U', 94: 'LP',
        104: 'LP', 105: 'U', 106: 'C', 108: 'LP', 110: 'T', 122: 'C', 126: 'C',
        128: '1', 130: 'LP', 132: 'LP', 134: 'LP', 136: '1', 138: 'LP',
        140: 'LP', 142: 'LP', 146: 'C', 150: 'U', 152: 'LP', 154: 'LP',
        156: 'LP', 158: 'LP', 160: '1', 162: 'LP', 164: 'LP', 168: '1',
        170: 'LP', 172: 'LP', 178: 'LP', 184: 'LP', 200: 'LP', 204: 'LP',
        232: 'LP'
    }

    rules_to_test = sorted(paper_results_full.keys())
    correct_count = 0

    print("开始验证论文表格中的所有唯一规则：")
    print("-" * 60)
    print(f"{'Rule':<10} | {'Paper Subclass':<20} | {'Our Code Output':<20} | {'Match'}")
    print("-" * 60)

    for rule in rules_to_test:
        # 增加试验次数以获得更可靠的结果
        classification = classify_rule(rule, num_trials=50, duration=3000)
        paper_subclass = paper_results_full.get(rule, "N/A")
        match = "✅" if classification == paper_subclass else "❌"
        if match == "✅":
            correct_count += 1
        print(f"{rule:<10} | {paper_subclass:<20} | {classification:<20} | {match}")

    print("-" * 60)
    accuracy = (correct_count / len(rules_to_test)) * 100
    print(f"总共测试规则数: {len(rules_to_test)}")
    print(f"匹配成功数: {correct_count}")
    print(f"准确率: {accuracy:.2f}%")

if __name__ == '__main__':
    # 注意：完整运行所有规则会非常耗时（可能需要一小时以上）
    verify_all_rules_from_paper()



开始验证论文表格中的所有唯一规则：
------------------------------------------------------------
Rule       | Paper Subclass       | Our Code Output      | Match
------------------------------------------------------------


0          | 1                    | 1                    | ✅


1          | LP                   | LP                   | ✅


2          | LP                   | LP                   | ✅


3          | LP                   | LP                   | ✅


...

178        | LP                   | LP                   | ✅


184        | LP                   | T                    | ❌


200        | LP                   | LP                   | ✅


204        | LP                   | LP                   | ✅


232        | LP                   | LP                   | ✅
------------------------------------------------------------
总共测试规则数: 89
匹配成功数: 66
准确率: 74.16%
